In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

### Permanentmagnet
$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
\DeclareMathOperator{\real}{real}
$$
Finde $ u \in H(\Curl)$ sodass 
$$
\int \mu^{-1} \Curl u \Curl v + 10^{-8} \int  u  v= \int M \Curl v \qquad  \forall \, v \in H(\Curl) \tag{1}
$$


Wobei der zweite Integralterm der Stabilität dient und die Werte für $\mu_r$ wie folgt angenommen werden:
* magnet: $\mu_r = 1$
* copper plate: $\mu_r = 1-6.4\cdot10^{-6}$
* iron plate: $\mu_r = 1000$
* air: $\mu_r = 1$ 


In [3]:
geo = CSGeometry()
box_size = 4
box = OrthoBrick(Pnt(-box_size,-box_size,-box_size), Pnt(box_size,box_size,box_size)).bc("outer")
magnet = Cylinder(Pnt(0.05,0,0), Pnt(0.05,0,1), 0.3) * \
            OrthoBrick (Pnt(-1,-1,-1),Pnt(1,1,1)).mat("magnet")

plate = OrthoBrick (Pnt(-1, 1,-1.5),Pnt(1,1.2,1.5)).mat('copper').maxh(0.1)

air = (box - magnet - plate).mat('air') 

geo.Add(plate)#, col=(0,1,0)) # what is col?
geo.Add(magnet)
geo.Add(air)#, transparent=True)

mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(5)
Draw (mesh)
# help (geo)
# help(box)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

In [10]:
fes = HCurl(mesh, order = 3, dirichlet="outer", nograds=True)
u,v = fes.TnT()
from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1 if mat== "magnet" else 1-6.4*1e-7 if mat=='copper' else 1000 if mat == 'iron' else 1
                            for mat in mesh.GetMaterials()]) 
a = BilinearForm(fes)
a += 1/(mu0*mur)*curl(u)*curl(v)*dx + 1e-6/(mu0*mur)*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((0,0,1)) * \
    CoefficientFunction([100 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])
f += mag*curl(v)*dx(mesh.Materials("magnet"))

gfu = GridFunction(fes)

with TaskManager():
    a.Assemble()
    f.Assemble()
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)

# help(c)
# help(fes)

iteration 0 error = 0.13567282543016937
iteration 1 error = 0.05473322722844279
iteration 2 error = 0.04438502193241739
iteration 3 error = 0.027011161170893104
iteration 4 error = 0.0257904700230619
iteration 5 error = 0.017863894939035896
iteration 6 error = 0.012876952609546239
iteration 7 error = 0.007108495240556475
iteration 8 error = 0.004363922338849532
iteration 9 error = 0.0027391175599099675
iteration 10 error = 0.0018865738552317041
iteration 11 error = 0.0013196573532276775
iteration 12 error = 0.0009264697631952405
iteration 13 error = 0.0006107677934272514
iteration 14 error = 0.00044131375854792806
iteration 15 error = 0.00030263062301543733
iteration 16 error = 0.00018608232376703717
iteration 17 error = 0.00012186644261898285
iteration 18 error = 8.404812877390286e-05
iteration 19 error = 5.851398774773702e-05
iteration 20 error = 4.072693408649215e-05
iteration 21 error = 2.7305117975917054e-05
iteration 22 error = 1.8433532239517095e-05
iteration 23 error = 1.294307

In [4]:
## magnetic flux
Draw (curl(gfu), mesh, "B-field", draw_surf=False)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

In [5]:
Draw (gfu, mesh, "vector-potential", draw_surf = False)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

In [11]:
## magnetic field
Draw (1/(mu0*mur)*curl(gfu), mesh, "H-field", draw_surf=False)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

### Harmonische Magnetisierung

Wird noch durch mit Spule umwickeltem Eisenkern ersetzt, hier mal die einfachere Schummelvariante zum Ausprobieren. 


Finde $ u \in H(\Curl)$ sodass 
$$
\int_{\Omega} \mu^{-1} \Curl u \Curl v + i\omega\sigma \int_{\Omega\setminus\text{magnet}}  u  v  + \text{reg}\int_{\text{magnet}} u v = \int_{\Omega} M \Curl v \qquad  \forall \, v \in H(\Curl) \tag{2}
$$

mit den Werten
* magnet: $\mu_r = 1, \sigma = 0$ 
* copper plate: $\mu_r = 1-6.4\cdot10^{-6}, \sigma = 58 \cdot 10^8$
* air: $\mu_r = 1, \sigma = 10^{-5}$ 

In [6]:
fes = HCurl(mesh, order = 3, dirichlet="outer", complex= True)
u,v = fes.TnT()
from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1 if mat== "magnet" else 1-6.4*1e-7 if mat=='copper' else  1
                            for mat in mesh.GetMaterials()]) 

sigma = CoefficientFunction( [0 if mat== "magnet" else 58*1e8 if mat=='copper' else 1e-5
                            for mat in mesh.GetMaterials()]) 
omega = Parameter(100)
a = BilinearForm(fes)
a += 1/(mu0*mur)*curl(u)*curl(v)*dx + 1e-6/(mu0*mur)*u*v*dx ('magnet') + 1j*omega*sigma*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((0,0,1)) * \
    CoefficientFunction( [100 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])
f += mag*curl(v)*dx(mesh.Materials("magnet"))

gfu = GridFunction(fes)

with TaskManager():
    a.Assemble()
    f.Assemble()
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat, tol= 1e-8)

iteration 0 error = 0.13556131064627025
iteration 1 error = 0.054692369259469326
iteration 2 error = 0.044383183794187576
iteration 3 error = 0.02994433264261015
iteration 4 error = 0.041193223997615235
iteration 5 error = 0.0383559395910652
iteration 6 error = 0.023720522145325423
iteration 7 error = 0.02516850231592143
iteration 8 error = 0.015260576695808473
iteration 9 error = 0.014610352281020502
iteration 10 error = 0.010237179184604114
iteration 11 error = 0.007673098592359207
iteration 12 error = 0.00639783537450731
iteration 13 error = 0.00484225694385623
iteration 14 error = 0.003525633417056888
iteration 15 error = 0.00282730922461315
iteration 16 error = 0.0022859717495004057
iteration 17 error = 0.0017364095213312596
iteration 18 error = 0.0016562321845170309
iteration 19 error = 0.0014770968994471945
iteration 20 error = 0.000934919887882861
iteration 21 error = 0.0008046307718560777
iteration 22 error = 0.00042064721044990437
iteration 23 error = 0.0004720449249512781
it

In [5]:
# drawu = GridFunction(fes)
## need equivalent to
# drawu.vec.data = (-1j*omega*exp(1j*omega*1)*gfu).data
# scene = Draw (drawu.real, mesh, "E-field", draw_surf=False)
# def slider_func(t): 
    ## need equivalent to
#     drawu.vec.data = (-1j*omega*exp(1j*omega*t)*gfu).data
#     scene.Redraw()
# interact(slider_func, x=widgets.FloatSlider(min=1, max=100, step=0.05, value=1))

t = 1
Draw ((-1j*omega*exp(1j*omega*t)*gfu).real, mesh, "E-field", draw_surf=False)


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

In [9]:
t = 1
Draw (sigma*(-1j*omega*exp(1j*omega*t)*gfu).real, mesh, "j", draw_surf=False)
## to see: clipping (0,1,0,-1)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-128-gc46d6894b', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2…

## Notizen für mich
nützliche links:

* http://www.numa.uni-linz.ac.at/Teaching/PhD/Finished/zaglmayr 
* https://de.wikipedia.org/wiki/Maxwell-Gleichungen
* https://ngsolve.org/docu/latest/i-tutorials/wta/maxwell.html
* https://ngsolve.org/docu/latest/i-tutorials/unit-2.3-hcurlhdiv/hcurlhdiv.html
* https://ngsolve.org/docu/latest/i-tutorials/unit-2.4-Maxwell/Maxwell.html
* https://www.asc.tuwien.ac.at/~schoeberl/wiki/lva/notes/maxwell.pdf
* https://www.numa.uni-linz.ac.at/Teaching/Bachelor/kagerer-bakk.pdf
* "allwissender Kreis" 

### Fragen

* Ist es möglich die äußere Box unsichtbar zu machen?
* Was ist `col` bei geo.Add(..)
* Was bringt `nograds` (lt Help 'Remove higher order gradients of H1 basis functions from HCurl FESpace')?
* slider function Problem
* Wie gehe ich mit leitenden Materialien um (Permittivität)? 
* mehr clipping planes? 
* impressed current density?

### Basics

#### Physikalische Größen

$E$ .. Elektrische Feldstärke (Stärke und Richtung eines elektrischen Feldes) \
$D$ .. Elektrische Flussdichte (Dichte der elektrischen Feldlinien in Bezug auf eine Fläche) \
$H$ .. Magnetische Felstärke \
$B$ .. Magnetische Flussdichte \
$M$ .. Magnetisierung \
$\rho$ .. Ladungsdichte (Ladung pro Volumen) \
$\sigma$ .. elektrische Leitfähigkeit \
$j$ .. Stromdichte (Strom pro durchflossene Fläche) 


#### Makroskopische Maxwelllgleichungen
(M1) $\Div D = \rho$ (Ladung ist Quelle des elektrischen Feldes) \
(M2) $\Div B = 0$ (keine magnetischen Ladungen) \
(M3) $\Curl E = -\partial_t B$ (Induktion)  \
(M4) $\Curl H = j + \partial_t D$ (Amper'sches Gesetz mit Verschiebungsstrom)

#### sonstige Zusammenhänge

(S1) $D = \epsilon E $ (evtl noch $+ P$ .. Polarisation, was ist das?) wobei $\epsilon = \epsilon_0 \epsilon_r$ (Permittivität),  $\epsilon_0 = 10^{-7} \cdot 8.8541878128$\
(S2) $H = \mu^{-1} B - M$ wobei $\mu = \mu_0 \mu_r$ (magnetische Permeabilität wenn im linearen Bereich), $\mu_0 =  4\pi10^{-7}$ \
(S3) $j = \sigma E +j_i$ ($j_i$ .. 'impressed current density', was ist das genau?)


#### Herleitung einer schwachen Formulierung
Einführen eines Vektorpotentials $A$ sodass $B = \Curl A$. 

Umformen:
$$
\begin{split}
\Curl(\mu^{-1} \Curl(A) - M) &= \Curl(\mu^{-1} B - M) \\
&\stackrel{\text{(S2)}}{=} \Curl(H) \\
&\stackrel{\text{(M4)}}{=} j + \partial_t D \\
&\stackrel{\text{(S1)}}{=} j + \partial_t \epsilon E  \\
&\stackrel{\text{(S3)}}{=} \sigma E + j_i + \partial_t \epsilon E 
\end{split}
$$


Ziel ist $A$ so zu wählen, dass $E = -\partial_t A$. Einzige Bedingung an $A$ war, dass $\Curl A = B$, dh jedes $\tilde{A}$ das sich von $A$ nur um ein Gradientenfeld unterscheidet erfüllt das genauso. Sei $\tilde{A}$ so dass $\Curl \tilde{A} = B$



Nach (M3) gilt $\Curl E = - \partial_t (\Curl \tilde{A})) = - \Curl( \partial_t \tilde{A})$. Äquivalent dazu $\Curl(E + \partial_t \tilde{A}) = 0$. Gibt Potential $\phi: \R^3 \rightarrow \R $ mit $\Grad \phi =  E+ \partial_t \tilde{A}$ wenn das Gebiet brav genug ist (sternförmig oder einfach zusammenhängend). Angenommen das ist der Fall dann gilt $E = \Grad \phi - \partial_t \tilde{A}$. Mit 
$$
A = - (\int_{t_0}^{t} \Grad \phi(s) ~ \text{ds} - \tilde{A})
$$ 
ist die gewünschte Anforderung erfüllt. Dh wir können annehmen $E = -\partial_t A$. 

Einsetzen ergibt
$$
\Curl(\mu^{-1} \Curl(A) - M) + \sigma \partial_t A +  \epsilon \partial_t^2A = j_i .
$$


Partielle Integration für curl 
$$
\int_\Omega \Curl u \cdot v = \int_\Omega u \cdot \Curl v - \int_{\partial\Omega} (u \times n) \cdot v
$$


Angenommen wir haben Dirichlet RB, dann ergibt sich die schwache Formulierung:

__________

Finde $A \in H(\Curl)$ sodass 
$$ 
\int \mu^{-1} \Curl A \Curl v +  \sigma \int \partial_t A v + \epsilon \int \partial_t^2 A v = \int j_i v + \int M \Curl v \qquad \forall v \in H(\Curl). 
$$
__________

#### Spezialfälle

Permanentmagnet:  $j_i = 0$ und  $E = 0$ $\Rightarrow$ (1) 

harmonische Magnetisierung:  $j_i = 0$ , $M(x,t) = \real(M(x)e^{i\omega t})$ und $A = \real(A(x)e^{i\omega t})$ $\Rightarrow$ (2) 
